In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.optim as optim
import time



class BasicBlock(nn.Module):
    def __init__(self, in_fea, out_fea, stride=1):
        super(BasicBlock, self).__init__()
        self.block=nn.Sequential(nn.Conv2d(in_fea, out_fea, 3, padding=1, stride=stride, bias=False),
                               nn.BatchNorm2d(out_fea),
                               nn.Dropout(0.1),
                               nn.ReLU(),
                               nn.Conv2d(out_fea, out_fea, 3, padding=1),
                               nn.BatchNorm2d(out_fea),
                               )
        b=[nn.Conv2d(in_fea, out_fea, 1)]
        if stride>1:
            b.append(nn.MaxPool2d(stride))
        self.shortcut=nn.Sequential(*b)
        
    def forward(self, x):
        x=self.block(x)+self.shortcut(x)
        return F.relu(x)  
        
class ResNet(nn.Module):
    def __init__(self, features, strides, num_class=10):
        super(ResNet, self).__init__()
        b = [BasicBlock(fea_in, fea_out, stri) for fea_in, fea_out, stri in zip(features[:-1], features[1:], strides)]
        self.layers=nn.Sequential(*b)
        self.last=nn.Linear(features[-1],num_class)
        
    def forward(self, x):
        x=self.layers(x)
        N,C,h,w=x.shape
        x=F.max_pool2d(x,(h,w))
        x=self.last(x.view(x.shape[0],-1))
        return x


def train(epoch, to_print=True):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    if to_print:
        print('epoch %d Train loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (epoch+1, train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print('        Test loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [5]:
X=np.load('Competition_Train_data_40000_dist.npy')
y=np.load('Competition_Train_label_40000_dist.npy')
X=X.reshape(-1,1,64,64)
I =np.arange(X.shape[0])
np.random.shuffle(I)
X_train=X[I[:int(0.75*X.shape[0])]]
y_train=y[I[:int(0.75*X.shape[0])]]
X_val=X[I[int(0.75*X.shape[0]):]]
y_val=y[I[int(0.75*X.shape[0]):]]
    
from torch.utils.data import Dataset, DataLoader
class get_data(Dataset):
    def __init__(self, data,label):
        self.data = torch.FloatTensor(data.astype('float'))
        self.label = torch.from_numpy(label).long()
        
    def __len__(self):
        return self.label.shape[0]
    
    def __getitem__(self, index):
        data_val = self.data[index]
        target = self.label[index]
        return data_val,target
    
batch_size = 100
trainset = get_data(X_train, y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
testset = get_data(X_val,y_val)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)


In [6]:
# train network
features=[1,32,32,64,64,120,300]
strides=[1,2]*3
num_class=16

net = ResNet(features, strides, num_class)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    net = net.cuda()
    print("using gpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(),lr=0.0001)

num_epochs = 15

for epoch in range(num_epochs):
    train(epoch)
    test(epoch)  

using gpu
epoch 1 Train loss: 0.666 | Acc: 80.527% (24158/30000)
        Test loss: 0.149 | Acc: 96.660% (9666/10000)
epoch 2 Train loss: 0.087 | Acc: 97.970% (29391/30000)
        Test loss: 0.055 | Acc: 98.570% (9857/10000)
epoch 3 Train loss: 0.035 | Acc: 99.273% (29782/30000)
        Test loss: 0.022 | Acc: 99.450% (9945/10000)
epoch 4 Train loss: 0.018 | Acc: 99.607% (29882/30000)
        Test loss: 0.012 | Acc: 99.790% (9979/10000)
epoch 5 Train loss: 0.011 | Acc: 99.803% (29941/30000)
        Test loss: 0.007 | Acc: 99.880% (9988/10000)
epoch 6 Train loss: 0.007 | Acc: 99.887% (29966/30000)
        Test loss: 0.024 | Acc: 99.240% (9924/10000)
epoch 7 Train loss: 0.006 | Acc: 99.833% (29950/30000)
        Test loss: 0.007 | Acc: 99.810% (9981/10000)
epoch 8 Train loss: 0.004 | Acc: 99.923% (29977/30000)
        Test loss: 0.005 | Acc: 99.870% (9987/10000)
epoch 9 Train loss: 0.003 | Acc: 99.960% (29988/30000)
        Test loss: 0.003 | Acc: 99.930% (9993/10000)
epoch 10 Train los

In [7]:
import numpy as np
Problems = np.load('Competition_Problems.npy')
print(Problems.shape)

(20000, 64, 384)


In [8]:
# segment data
Nex = Problems.shape[0]
dseg = 3
thr = 0.0001
X = np.sum(Problems,axis=1,keepdims=True)/np.sum(np.sum(Problems,axis=1,keepdims=True),axis=2,keepdims=True)

First = np.zeros((Nex,1,64,64))
Second = np.zeros((Nex,1,64,64))
Third = np.zeros((Nex,1,64,64))
for ex in range(Nex):
    m = np.array(np.where(X[ex,0,64-dseg:2*64+dseg]>thr)).T
    m2 = int(np.average(m+64-dseg))
    First[ex,0,:,:] = Problems[ex,:,int(m2-32):int(m2+32)]
    
    m = np.array(np.where(X[ex,0,4*64-dseg:5*64+dseg]>thr)).T
    m5 = int(np.average(m+4*64-dseg))
    Second[ex,0,:,:] = Problems[ex,:,int(m5-32):int(m5+32)]
    
    Third[ex,0,:,:] = Problems[ex,:,int(384-64):384]



# make predictions
First=np.reshape(First,(-1,1,64,64))
Second=np.reshape(Second,(-1,1,64,64))

pred_First=np.zeros(First.shape[0])
pred_Second=np.zeros(Second.shape[0])
batch_size = 100

for i in range(int(200/1)):
    syms=torch.FloatTensor(First[i*batch_size:(i+1)*batch_size].astype('float')).to(device)
    T=net(syms).detach().cpu().numpy()    
    pred_First[i*batch_size:(i+1)*batch_size]=np.argmax(T,1)
    
    syms=torch.FloatTensor(Second[i*batch_size:(i+1)*batch_size].astype('float')).to(device)
    T=net(syms).detach().cpu().numpy()
    pred_Second[i*batch_size:(i+1)*batch_size]=np.argmax(T,1)
    
# computing the score according to the grading rules 
A=(pred_First<4)
B=(pred_First<7)*(pred_First>3)
C=(pred_First>0)*A
D7=(pred_Second-pred_First==7)
D3=(pred_Second-pred_First==3)
D12=(pred_Second-pred_First==12)
D_3=(pred_Second-pred_First==-3)
Score=(A*D7*10+B*D_3*10+A*D12*5+C*D3*2)*(np.sum(np.sum(Third[:,:,10:64],-1),-1)>200).reshape(-1,)
Score += (A*D7*9+B*D_3*9+A*D12*5+C*D3*1)*(np.sum(np.sum(Third[:,:,10:64],-1),-1)<200).reshape(-1,)

# Save the score as my_submission, which can be uploaded for the submission.
np.save('joe_submission2.npy',Score)
print('Done! The first 10 problems have predicted scores:', Score[:10])

Done! The first 10 problems have predicted scores: [ 9  1  0  0  0  0  1  5 10  1]
